<a href="https://colab.research.google.com/github/nrkfeller/YCBS_notes/blob/master/C8_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploy Keras Model to the Cloud
1. Download data
2. Hot Encode Categorical Columns
3. Develop Model (use: ```RMSprop(lr=, rho=, epsilon=, decay=)```)
3. Create Storage Bucket (should already exist)
4. Authenticate on Google Cloud Platform
5. Deploy your model using the ```gcloud``` tool

### Download Data

In [0]:
!wget https://storage.googleapis.com/nicksdemobucket/bank.csv

In [0]:
import pandas as pd
df = pd.read_csv('bank.csv', sep=";")
df.head()

### Hot Encode Categorical Columns

In [0]:
# Get all categorical columns
categorical = df.select_dtypes(include='object').columns
print(categorical)

In [0]:
# Hot encode all categorical columns
for i in categorical:
  new_cols = pd.get_dummies(df[i], prefix=i)
  df = df.drop(i, axis=1)
  df = df.join(new_cols)
  
df.head()

In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.001)

X_train = train.drop(['y_no', 'y_yes'], axis=1).values
y_train = train[['y_no', 'y_yes']].values

X_test = test.drop(['y_no', 'y_yes'], axis=1).values
y_test = test[['y_no', 'y_yes']].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Develop Model
* Use ```RMSprop(lr=, rho=, epsilon=, decay=)```
* Plot accuracy and loss over time

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.train import AdamOptimizer
import tensorflow as tf

### Create Storage

In [0]:
PROJECT_ID = "" #@param {type:"string"}

# This must absolutely be a GLOBALLY UNIQUE name
BUCKET_NAME = "" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}

! gcloud config set project $PROJECT_ID
! echo $PROJECT_ID

### Authenticate on Google Cloud Platform
* Click on link and enter secret key

In [0]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [0]:
# Create Bucket
! gsutil mb -p $PROJECT_ID -l $REGION gs://$BUCKET_NAME
# Display what is in the bucket
!gsutil ls -al gs://$BUCKET_NAME

In [0]:
JOB_DIR = 'gs://' + BUCKET_NAME + '/modelfolder'

### Export Model to Google Cloud Storage

In [0]:
export_path = tf.contrib.saved_model.save_keras_model(model, JOB_DIR + '/keras_export')
print("Model exported to: ", export_path)

### Deploy Model from GCS

### Test your model with your test set

In [0]:
# import numpy as np
# for i in X_test:
#   #print("%a" % i)
#   print(i, sep=',')

with open('test.json', 'w') as f:
  for item in X_test:
    item = list(item)
    f.write("%s\n" % item)

In [0]:
! cat test.json